In [1]:
import dill
dill.load_session('latest-run.db') #this can load a saved python session so I don't need to rerun computationally expensive cells

/home/sylphrena/anaconda3/envs/CLASSE/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%autosave 300 
#autosaves code every five minutes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matminer.datasets import get_available_datasets, load_dataset, get_all_dataset_info
from matminer.featurizers.conversions import StrToComposition

#try pymatminer

# load supercon databse and metadata
# print(get_all_dataset_info("superconductivity2018")) # metadata
data = pd.DataFrame(pd.read_csv('supercon.csv'))
data.head()

stc = StrToComposition()
composition = stc.featurize_dataframe(data,'name', ignore_errors=True)

#features will be made with matminer, target is Tc
composition.head()

Autosaving every 300 seconds


StrToComposition: 100%|██████████| 16414/16414 [00:04<00:00, 3661.56it/s]


,name,Tc,composition
0,Ba0.4K0.6Fe2As2,31.20,"(Ba, K, Fe, As)"
1,Ca0.4Ba1.25La1.25Cu3O6.98,40.10,"(Ca, Ba, La, Cu, O)"
2,Mo0.39Ru0.61,6.90,"(Mo, Ru)"
3,Tm4Os6Sn19,1.10,"(Tm, Os, Sn)"
4,Nd1Bi0.99Pb0.01S2F0.3O0.7,4.85,"(Nd, Bi, Pb, S, F, O)"


### Attempting to create feature manually with matminer featurizers
Code doesn't fully run yet, I'm just testing.

In [3]:
from matminer.featurizers.composition import ElectronAffinity, ElementProperty, TMetalFraction, Stoichiometry, BandCenter, OxidationStates, IonProperty, ElectronegativityDiff, AtomicOrbitals, ValenceOrbital, AtomicPackingEfficiency, CohesiveEnergy, CohesiveEnergyMP
#WARNING - this will take a while (>2 hours) to run!

#these are all the Magpie features. You can choose fewer if you want.
features = ['Number', 'MendeleevNumber', 'AtomicWeight', 'MeltingT', 
            'Column', 'Row', 'CovalentRadius', 'Electronegativity', 
            'NsValence', 'NpValence', 'NdValence', 'NfValence', 'NValence', 
            'NsUnfilled', 'NpUnfilled', 'NdUnfilled', 'NfUnfilled', 'NUnfilled', 
            'GSvolume_pa', 'GSbandgap', 'GSmagmom', 'SpaceGroupNumber']

ea_prop = ElementProperty(data_source='magpie', features=features, stats=['mean']) # can add 'minimum', 'maximum', 'range', 'avg_dev', 'mode' to stats
ea_prop_data = ea_prop.featurize_dataframe(composition, 'composition', ignore_errors=True)

el_prop = ElementProperty(data_source='magpie', features=features, stats=['mean'])
el_prop_data = el_prop.featurize_dataframe(composition, 'composition', ignore_errors=True)
#pymatgen
#feild, goals, stuff we are doing to achive goals - presentation
ea_aff = ElectronAffinity()
ea_aff_data = ea_aff.featurize_dataframe(composition, 'composition', ignore_errors=True)

met_frac = TMetalFraction()
met_frac_data = met_frac.featurize_dataframe(composition, 'composition', ignore_errors=True)

stoich = Stoichiometry()
stoich_data = stoich.featurize_dataframe(composition, 'composition', ignore_errors=True)

band_center = BandCenter()
band_center_data = band_center.featurize_dataframe(composition, 'composition', ignore_errors=True)

ox_states = OxidationStates()
ox_states_data = ox_states.featurize_dataframe(composition, 'composition', ignore_errors=True)

ion_prop = IonProperty()
ion_prop_data = ion_prop.featurize_dataframe(composition, 'composition', ignore_errors=True)

en_diff = ElectronegativityDiff()
en_diff_data = en_diff.featurize_dataframe(composition, 'composition', ignore_errors=True)

atom_orbitals = AtomicOrbitals()
atom_orbitals_data = atom_orbitals.featurize_dataframe(composition, 'composition', ignore_errors=True)

val_orbitals = ValenceOrbital()
val_orbitals_data = val_orbitals.featurize_dataframe(composition, 'composition', ignore_errors=True)

atom_pack_eff = AtomicPackingEfficiency()
atom_pack_eff_data = atom_pack_eff.featurize_dataframe(composition, 'composition', ignore_errors=True)

cohesive_en = CohesiveEnergy()
cohesive_en_data = cohesive_en.featurize_dataframe(composition, 'composition', ignore_errors=True)

cohesive_en_mp = CohesiveEnergyMP()
cohesive_en_mp_data = cohesive_en_mp.featurize_dataframe(composition, 'composition', ignore_errors=True)


Stoichiometry: 100%|██████████| 16414/16414 [00:16<00:00, 1007.90it/s]]

ElectronegativityDiff: 100%|██████████| 16414/16414 [00:14<00:00, 1098.99it/s]

CohesiveEnergyMP: 100%|██████████| 16414/16414 [14:27<00:00, 18.93it/s]


In [4]:
all_feat = ea_aff_data
for featurizer in [el_prop_data, met_frac_data, stoich_data, band_center_data, ox_states_data, ion_prop_data, en_diff_data, atom_orbitals_data, val_orbitals_data, atom_pack_eff_data, cohesive_en_data, cohesive_en_mp_data]:
    all_feat = pd.merge(all_feat, featurizer, how="left")
all_feat.to_csv('supercon_feat.csv')

## Initial Testing of Algorithms

In [5]:
# from automatminer import MatPipe #automatminer uses a import that was removed in scikit-learn  0.24. I am using version 0.23
# I can't get this to work :(

from sklearn.model_selection import train_test_split

# test train split, with shuffle and random state for reproducibility 
train_data, test_data, train_target, test_target = train_test_split(data['composition'], data['Tc'], test_size=0.15, random_state=43, shuffle=True)



KeyError: 'composition'

In [6]:
dill.dump_session('latest-run.db') #this can dump a python session so I can resume later, after restarts and such